# Loading the state of the art v1 model

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import transforms 

In [2]:
# suppress datajoint asking for credentials

os.environ["DJ_USER"] = ""
os.environ["DJ_HOST"] = ""
os.environ["DJ_PASS"] = ""

In [3]:
# get model
from nnvision.models.trained_models.v1_data_driven import v1_data_driven_sota as model
from nnvision.models.trained_models.v1_task_fine_tuned import v1_convnext, v1_convnext_ensemble

Connecting @:3306
datajoint connection not established, skipping model imports from nnfabrik tables


## Model predictions

In [7]:
# set model evaluation mode - as default, it is being instantiated in "train" mode
model.eval().cuda();

In [8]:
# the model was trained with images of size 93,93, and can only deal with these input sizes
# the model expects z-scored images, with a range of [-1.74, 1.98]
example_input = torch.zeros(1,1,93,93).cuda()

In [9]:
with torch.no_grad():
    responses = model(example_input, data_key="all_sessions")

In [10]:
print(responses)

tensor([[1.1181e+00, 1.6059e+00, 1.4834e-01, 3.7733e-01, 2.4578e+00, 1.3335e+00,
         3.0964e+00, 1.3711e+00, 1.6154e+00, 4.1894e-01, 1.2986e+00, 2.1720e-01,
         6.0975e-01, 3.2478e-01, 1.1298e+00, 4.4063e-01, 5.2015e+00, 2.8679e-01,
         3.0494e-01, 1.2427e-01, 1.5341e+00, 1.4822e-01, 1.9858e-01, 9.8038e-01,
         5.4465e-02, 1.5439e-01, 2.3378e-01, 7.7243e-02, 7.8714e-03, 1.7342e-01,
         2.4274e-01, 1.2975e-02, 4.0584e-01, 1.7045e-02, 1.0214e+00, 5.7988e-01,
         4.1030e-02, 8.3904e-01, 2.6028e+00, 2.3915e+00, 1.0458e+00, 4.9367e+00,
         2.0116e+00, 1.2702e+00, 1.1700e+00, 5.2850e-01, 2.8497e-01, 2.8380e-01,
         4.0336e-01, 7.3382e-01, 1.8786e+00, 1.1649e-02, 1.6225e-02, 2.2927e+00,
         7.7852e-01, 8.4177e-02, 1.5631e+00, 3.9143e-01, 3.8841e-01, 1.0009e+00,
         5.2198e-01, 7.4958e-01, 3.5369e-01, 1.7155e+00, 2.1371e+00, 8.4717e-01,
         1.2425e+00, 1.2380e+00, 2.5236e-01, 1.7533e-01, 2.9741e+00, 8.6438e-01,
         4.6480e-01, 6.8596e

In [11]:
# 458 neurons
responses.shape

torch.Size([1, 458])

# Load Ensemble models

In [13]:
# get ensemble models, these are 5 models each, with the individual responses being averaged
from nnvision.models.trained_models.v1_data_driven import v1_data_driven_sota_ensemble_model, v1_data_driven_sota_ensemble_model_2

In [14]:
# prints all 5 models
v1_data_driven_sota_ensemble_model

EnsembleModel(Encoder(
  (core): SE2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 32, kernel_size=(24, 24), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(8, 8), dilation=(2, 2), groups=32, bias=False)
          (out_depth_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
        (seg_ex_block): SQ_EX_Block(
          (se): Seq

In [16]:
# the individual model
v1_data_driven_sota_ensemble_model.members[0]

Encoder(
  (core): SE2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 32, kernel_size=(24, 24), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(8, 8), dilation=(2, 2), groups=32, bias=False)
          (out_depth_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
        (seg_ex_block): SQ_EX_Block(
          (se): Sequential(
     

In [17]:
# can be treated like a normal model
v1_data_driven_sota_ensemble_model.eval().cuda();

In [18]:
with torch.no_grad():
    responses = v1_data_driven_sota_ensemble_model(example_input, data_key="all_sessions")

In [19]:
responses

tensor([[1.0428e+00, 1.5648e+00, 1.5690e-01, 3.5774e-01, 2.4855e+00, 1.3269e+00,
         3.0897e+00, 1.3386e+00, 1.7092e+00, 3.9005e-01, 1.2723e+00, 2.1644e-01,
         5.8329e-01, 3.3417e-01, 1.0781e+00, 4.4573e-01, 5.1325e+00, 2.7602e-01,
         2.9167e-01, 1.1466e-01, 1.4511e+00, 1.4938e-01, 1.7244e-01, 9.2998e-01,
         5.2448e-02, 1.4952e-01, 2.3755e-01, 8.9157e-02, 8.4780e-03, 1.7039e-01,
         2.4126e-01, 9.7699e-03, 4.0134e-01, 2.0930e-02, 1.0020e+00, 5.1652e-01,
         3.9660e-02, 8.5605e-01, 2.5323e+00, 2.3878e+00, 1.0406e+00, 4.8332e+00,
         1.9719e+00, 1.2718e+00, 1.1525e+00, 5.3854e-01, 2.8251e-01, 2.9873e-01,
         4.1204e-01, 6.9227e-01, 1.8559e+00, 1.4940e-02, 1.9636e-02, 2.2801e+00,
         7.5787e-01, 8.5021e-02, 1.5402e+00, 4.1918e-01, 4.0678e-01, 1.0638e+00,
         5.3358e-01, 7.9757e-01, 3.5897e-01, 1.6919e+00, 2.1820e+00, 8.5010e-01,
         1.2212e+00, 1.1868e+00, 2.2640e-01, 1.5712e-01, 2.9295e+00, 8.7799e-01,
         4.5890e-01, 6.4538e

# the ConvNext

In [4]:
v1_convnext

EncoderShifter(
  (core): ConvNextCore(
    (features): Sequential(
      (backbone): ConvNextV2(
        (model): ConvNextV2Model(
          (embeddings): ConvNextV2Embeddings(
            (patch_embeddings): Conv2d(3, 40, kernel_size=(4, 4), stride=(4, 4))
            (layernorm): ConvNextV2LayerNorm()
          )
          (encoder): ConvNextV2Encoder(
            (stages): ModuleList(
              (0): ConvNextV2Stage(
                (downsampling_layer): Identity()
                (layers): Sequential(
                  (0): ConvNextV2Layer(
                    (dwconv): Conv2d(40, 40, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=40)
                    (layernorm): ConvNextV2LayerNorm()
                    (pwconv1): Linear(in_features=40, out_features=160, bias=True)
                    (act): GELUActivation()
                    (grn): ConvNextV2GRN()
                    (pwconv2): Linear(in_features=160, out_features=40, bias=True)
                    (drop_path